In [1]:
import os
import cv2
import imutils
#import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
from PIL import Image

import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F
import torchvision.transforms as T
import torchvision.models as models
#from google.colab import output

# import albumentations as A
# from albumentations.core.transforms_interface import ImageOnlyTransform

from efficientnet_pytorch import EfficientNet

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 디바이스 설정
print(device)

cuda:0


In [2]:
def add_noise(inputs):
    noise = torch.randn_like(inputs)
    
    return inputs + noise

## Dataset 정의

In [3]:
class CustomDataset(D.Dataset):
    """
    path = {BASE_PATH,DATA_DIR1, DATA_DIR2 ,CSV_PATH}
    Return: pytorch custome dataset format 
    """
    def __init__(self, path, data, label, transform=None):
        self.path = path
        self.data = data
        self.label = label
        self.transform = transform
#         self.diagonal_reverse = diagonal_reverse
#         self.add_noise = add_noise
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = Image.open(self.path + self.data[idx])
        label = self.label[idx]
            
        if self.transform:
            image = self.transform(image)
#             image = self.diagonal_reverse(image)
#             image = add_noise(image)
        
        return image, label

## Model 정의

In [4]:
class My_model(nn.Module):
    def __init__(self, pretrained):
        super(My_model, self).__init__()
        self.conv2d = nn.Conv2d(1, 3, 3, stride=1)
        #self.pretrained = models.resnet50()
        
        self.pretrained = pretrained
        self.FC = nn.Linear(1000, 26)

    def forward(self, x):
        # resnet의 입력은 [3, N, N]으로
        # 3개의 채널을 갖기 때문에
        # resnet 입력 전에 conv2d를 한 층 추가
        x = F.relu(self.conv2d(x))

        
        x = F.relu(self.pretrained(x))

        # 마지막 출력에 nn.Linear를 추가
        # multilabel을 예측해야 하기 때문에
        # softmax가 아닌 sigmoid를 적용
        x = torch.sigmoid(self.FC(x))
        return x

In [5]:
def save(state, SAVE_DIR, epoch, model, optimizer):
    with open(SAVE_DIR + state +".path.tar", "wb") as f:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()},
            f)

In [6]:
base_path = 'D:/dacon/2nd/'
namelist = os.listdir(base_path + 'dirty_mnist_2nd/')
# namelist = os.listdir(base_path)
labels = pd.read_csv(base_path + "dirty_mnist_2nd_answer.csv").to_numpy()[:, 1:]

save_dir = './save_file/'
os.makedirs(save_dir, exist_ok=True)

BATCH_SIZE = 24

In [7]:
# transformer = A.Compose([
#     A.RandomCrop(width=128, height=128),
#     A.HorizontalFlip(p=1),
#     A.CenterCrop(height=128, width=128),
#     A.Rotate(),
#     A.Cutout(
#     num_holes=10,
#     max_h_size=20,
#     max_w_size=20,
#     fill_value=0,
#     always_apply=True,
#     p=0.5,)
# ])

# test_transforms = A.Compose([
#     A.RandomCrop(width=128, height=128),
#     A.HorizontalFlip(p=1),
#     A.CenterCrop(height=128, width=128),
#     A.Rotate(),
#     A.Cutout(
#     num_holes=10,
#     max_h_size=20,
#     max_w_size=20,
#     fill_value=0,
#     always_apply=True,
#     p=0.5,)
# ])

# # train_dataset = aug_random_imshow(train_img_paths, transformer)
# # aug_random_imshow(val_img_paths, transformer)

# dataset = CustomDataset(base_path+'dirty_mnist_2nd/', namelist, labels, transformer)

# train_dataset, val_dataset = D.random_split(dataset, [len(dataset) - int(len(dataset) * 0.1), int(len(dataset) * 0.1)])

## Augmentation

In [8]:
transformer = T.Compose([
#     T.RandomCrop(128,128),
#     T.RandomRotation(2.8),
#     T.RandomHorizontalFlip(),
#     T.CenterCrop(10),
#     T.RandomVerticalFlip(),
    T.ToTensor(),
    T.Normalize((0.1307,), (0.3081,)),
    T.RandomRotation(60, expand=False),
    T.RandomAffine(30)
    #AddGaussianNoise(0., 1.)
])

test_transforms = T.Compose([
    T.ToTensor(),
    T.Normalize((0.1307,), (0.3081,)),
    T.RandomRotation(60, expand=False),
    T.RandomAffine(30)
    #AddGaussianNoise(0., 1.)
])

dataset = CustomDataset(base_path+'dirty_mnist_2nd/', namelist, labels, transformer)
train_dataset, val_dataset = D.random_split(dataset, [len(dataset) - int(len(dataset) * 0.1), int(len(dataset) * 0.1)])

In [9]:
train_dataloader =  torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
val_dataloader =  torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

In [10]:
# os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [11]:
# 모델 선언
pretrained = EfficientNet.from_pretrained('efficientnet-b3')
b4_model = My_model(pretrained)
model = nn.DataParallel(b4_model)
model.to(device)# gpu에 모델 할당

# 훈련 옵션 설정
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
#                                             step_size = 10,
#                                             gamma = 0.85)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max=0.1,eta_min=0.0001)
criterion = torch.nn.BCELoss()

Loaded pretrained weights for efficientnet-b3


In [12]:
# 훈련 시작
total_step = len(train_dataloader)
best_val_acc = 0
EPOCH = 50
for epoch in range(EPOCH):
    train_acc_list = []
    running_loss = 0
    
    model.train()
    for i, (images, labels) in tqdm(enumerate(train_dataloader)):
        images = images.type(torch.FloatTensor).to(device)
        labels = labels.type(torch.FloatTensor).to(device)
        
        optimizer.zero_grad()

        probs= model(images)
        loss = criterion(probs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        probs  = probs.cpu().detach().numpy()
        labels = labels.cpu().detach().numpy()
        preds = probs > 0.75
        batch_acc = (labels == preds).mean()
        train_acc_list.append(batch_acc)
    
    train_acc = np.mean(train_acc_list)
    print(f'Epoch [{epoch+1}/{EPOCH}], Step [{i+1}/{total_step}], Loss: {running_loss/total_step}, Acc {train_acc}')

    model.eval()
    valid_acc_list = []
    with torch.no_grad():
        correct = 0
        total = 0

        for images, labels in val_dataloader:
            images = images.type(torch.FloatTensor).to(device)
            labels = labels.type(torch.FloatTensor).to(device)

            probs = model(images)
            valid_loss = criterion(probs, labels)

            probs  = probs.cpu().detach().numpy()
            labels = labels.cpu().detach().numpy()
            preds = probs > 0.75
            batch_acc = (labels == preds).mean()
            valid_acc_list.append(batch_acc)
            
        val_acc = np.mean(valid_acc_list)
        print(f'Validation acc: {val_acc}')

    lr_scheduler.step()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        save('best', save_dir, epoch, model, optimizer)
    save('last', save_dir, epoch, model, optimizer)

1875it [10:10,  3.07it/s]


Epoch [1/50], Step [1875/1875], Loss: 0.6854681624094645, Acc 0.5397538461538461
Validation acc: 0.5412275024654832


1216it [06:34,  3.09it/s]


KeyboardInterrupt: 

###### Test

In [ ]:
test_namelist = os.listdir(base_path + 'test_dirty_mnist_2nd/')
test_labels = pd.read_csv(base_path + "sample_submission.csv").to_numpy()[:, 1:]

test_transforms = T.Compose([
    T.ToTensor(),
    T.Normalize((0.1307,), (0.3081,)),
    T.RandomRotation(60, expand=False),
    T.RandomAffine(30)
    #AddGaussianNoise(0., 1.)
])

test_dataset = CustomDataset(base_path+'/test_dirty_mnist_2nd/', test_namelist, test_labels, test_transforms)
test_dataloader =  torch.utils.data.DataLoader(test_dataset, shuffle=False)

In [ ]:
# model.load_state_dict(torch.load('save_file/best.path.tar'))
model.eval()
prediction_list = []
with torch.no_grad():
    for images, labels in tqdm(test_dataloader):
        images = images.type(torch.FloatTensor).to(device)
        labels = labels.type(torch.FloatTensor).to(device)

        probs = model(images)
        
        probs = probs.cpu().detach().numpy()
        preds = probs > 0.75
        prediction_list.append(preds[0].astype(np.int))

In [ ]:
file_name = 'eff_epoch50_batch24_Rotate_Affine_dead'

test_labels_DF = pd.read_csv(base_path + "sample_submission.csv")
test_labels_DF.iloc[:, 1:] = prediction_list
test_labels_DF.to_csv(file_name +'.csv', index=False)